# Connect to the cluster

In [ ]:
import socket
on_io = socket.gethostname() == 'io'
if on_io and 0:
    import hpc05, hpc05_monitor
    client = hpc05.connect.start_remote_and_connect(200, folder='~/two_dim_majoranas')[0]
    max_usage_task = hpc05_monitor.start(client)
else:
    # No not connect to the cluster in CI for example.
    client = None

## Numerics imports

In [ ]:
from functools import partial
from copy import copy
import numpy as np
import adaptive
import sns_system, kwant, spectrum
adaptive.notebook_extension()

import pathlib
pathlib.Path('thesis_report/figures').mkdir(parents=True, exist_ok=True) 

## Plotting imports

In [ ]:
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

class HistogramNormalize(matplotlib.colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        matplotlib.colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                       len(self.sorted_data))
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm

golden_mean = (np.sqrt(5) - 1) / 2  # Aesthetic ratio
inches_per_pt = 1.0 / 72.27  # Convert pt to inches
fig_width_pt = 6.2/inches_per_pt  # Columnwidth
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean  # height in inches
fig_size = [fig_width, fig_height]

params = {
          'backend': 'ps',
          'axes.labelsize': 13,
          'font.size': 13,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 100 if on_io else 300,
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}', r'\usepackage{xfrac}'])

## Parameter and system definitions

In [ ]:
import scipy.constants
import cmath

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle=26, g_factor_left=0, g_factor_right=0,
                 mu=10,
                 alpha_middle=20, alpha_left=0, alpha_right=0,
                 Delta_left=1, Delta_right=1,
                 B_x=0,
                 B_y=0,
                 B_z=0,
                 phase=np.pi,
                 T=0.0,
                 V=0)

default_params = dict(**constants,
              **params_raw)

default_syst_pars = dict(
    L_m=200,
    L_x=1300,
    L_sc_up=200,
    L_sc_down=200,
    z_x=1300,
    z_y=0,
    a=10,
    shape='parallel_curve',
    transverse_soi=True,
    mu_from_bottom_of_spin_orbit_bands=True,
    k_x_in_sc=True,
    wraparound=True,
    current=False,
    ns_junction=False)

# Figure 1. Bandstructures

In [ ]:
kx = np.linspace(-2.5, 2.5, 200)
Ekin = lambda k: np.array(k)**2
Esoi = lambda k, alpha: np.array([(np.array(k)+alpha)**2, (np.array(k)-alpha)**2]).T
Ezm_x  = lambda k, Ez: np.array([np.array(k)**2 - Ez, np.array(k)**2 + Ez]).T
Ezm_y  = lambda k, Ez: np.array([np.array(k)**2 - Ez, np.array(k)**2 + Ez]).T
def Hzsoi(k, alpha, Ez):
    return np.array([[k**2, -1j*k*alpha + Ez], [1j*k*alpha + Ez, k**2]])

def Hzsoi(k, alpha, Ez_x, Ez_y, Ez_z):
    return np.array([[k**2 + Ez_z, -1j*k*alpha + Ez_x -1j*Ez_y], [1j*k*alpha + Ez_x + 1j*Ez_y, k**2 - Ez_z]])

def Ezsoi(k, alpha, Ez_x, Ez_y, Ez_z):
    return [np.linalg.eigvalsh(Hzsoi(_k, alpha, Ez_x, Ez_y, Ez_z)) for _k in k] 

In [ ]:
tick_opts = {'axis': 'both', 'bottom':False, 'left':False, 'labelbottom':False, 'labelleft':False}
fig, axs_mat = plt.subplots(2, 2, figsize=(.5*1.5*1*fig_width, .5*1.5*1.5*fig_height))
axs_list = (*axs_mat[0], *axs_mat[1])
ax0, ax1, ax2, ax3 = axs_list

ax0.plot(kx, Ezsoi(kx, 0, 0, 0, 0), label=rf'kinetic');
ax1.plot(kx, Ezsoi(kx, 2, 0, 0, 0), label=rf'kinetic');
ax2.plot(kx, Ezsoi(kx, 0, .5, 0, 0), label=rf'kinetic');
ax3.plot(kx, Ezsoi(kx, 2, .5, 0, 0), label=rf'kinetic');
ax3.set_prop_cycle(None) 
ax3.plot(kx, Ezsoi(kx, 2, .0, .5, 0),'--', label=rf'kinetic');

[ax.tick_params(**tick_opts) for ax in axs_list];
[ax.set_xlabel('$k$')  for ax in [ax2, ax3]];
[ax.set_ylabel('$E$')  for ax in [ax3, ax1]];
[ax.yaxis.set_label_position('right')  for ax in axs_list];

pt_top_left = np.array([.4, 1.05])
pt_left_bottom = np.array([.6, 1.05])

arrow = matplotlib.patches.FancyArrow(
    pt_top_left[0], pt_top_left[1], pt_top_right[0]-pt_top_left[0], pt_top_right[1]-pt_top_left[1],
    transform=fig.transFigure, width=0.001, length_includes_head=True, head_width=0.03, color='k')

fig.text((pt_top_left + pt_top_right)[0]/2, (pt_top_left + pt_top_right)[1]/2 + .05, r'$+ H_\text{SOI}$', ha='center', va='center')

pt_left_top = np.array([-0.04, .6])
pt_left_bottom = np.array([-0.04, .4])

arrow2 = matplotlib.patches.FancyArrow(
    pt_left_top[0], pt_left_top[1], pt_left_bottom[0]-pt_left_top[0], pt_left_bottom[1]-pt_left_top[1],
    transform=fig.transFigure, width=0.001, length_includes_head=True, head_width=0.03, color='k')

fig.text((pt_left_top + pt_left_bottom)[0]/2-.05, (pt_left_top + pt_left_bottom)[1]/2, r'$+ H_\text{Zeeman}$', va='center', ha='center', rotation='vertical')

# fig.text(-.05, 1.05, r'$H_\text{kin}$', va='center', ha='center')
fig.text(.25, .925, r'\noindent$H_\text{kin}$', va='top', ha='center')
fig.text(.75, .925, r'\noindent$H_\text{kin} + \\H_\text{SOI}$', va='top', ha='center')
fig.text(.25, .475, r'\noindent$H_\text{kin} + \\H_\text{Zeeman}$', va='top', ha='center')
fig.text(.75, .475, r'\noindent$H_\text{kin} + \\H_\text{SOI}+ \\H_\text{Zeeman}$', va='top', ha='center')


# 5. Add patch to list of objects to draw onto the figure
fig.patches.append(arrow)
fig.patches.append(arrow2)

fig.tight_layout(pad=0.1)
plt.savefig('thesis_report/figures/simple_terms.pdf', bbox_inches='tight')
plt.show()